<a href="https://colab.research.google.com/github/Loga19818eeanvesh/YOLO_ObjectDetection/blob/main/YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow --upgrade
!pip install keras --upgrade

     |████████████████████████████████| 462 kB 4.9 MB/s 


In [2]:
import tensorflow as tf 
print(tf.__version__)

2.8.0


In [12]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D,Lambda,BatchNormalization,LeakyReLU,Add,InputLayer,MaxPool2D,Softmax,GlobalAveragePooling2D,Concatenate
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.regularizers import l2

In [4]:
class ConvBatchNormRelu(Model):
  def __init__(self,noof_filters,kernel_size):
    super().__init__()
    self.conv2d = Conv2D(filters=noof_filters, kernel_size=kernel_size,padding='same',kernel_regularizer=l2(5e-4))
    self.batch_norm = BatchNormalization()
    self.relu_activation = LeakyReLU(alpha=0.1)
    
  def call(self,inputs):
    x = self.conv2d(inputs)
    x = self.batch_norm(x)
    x = self.relu_activation(x)

    return x
  


In [5]:
class BottleNeckBlock(Model):
  def __init__(self,noof_filters,n):
    super().__init__()
    self.n=n
    self.conv_3by3_1 = ConvBatchNormRelu(noof_filters=noof_filters,kernel_size=3)
    self.conv_1by1_1 = ConvBatchNormRelu(noof_filters=noof_filters/2,kernel_size=1)
    self.conv_3by3_2 = ConvBatchNormRelu(noof_filters=noof_filters,kernel_size=3)
    if n==5:
      self.conv_1by1_2 = ConvBatchNormRelu(noof_filters=noof_filters/2,kernel_size=1)
      self.conv_3by3_3 = ConvBatchNormRelu(noof_filters=noof_filters,kernel_size=3)
    
  def call(self,inputs):
    x = self.conv_3by3_1(inputs)
    x = self.conv_1by1_1(x)
    x = self.conv_3by3_2(x)
    if self.n==5:
      x = self.conv_1by1_2(x)
      x = self.conv_3by3_3(x)

    return x
    

In [6]:
class YoloClassifier(Model):
  def __init__(self,input_shape=(None,224,224,3),noof_classes=1000):
    super().__init__()
    self.input_layer = InputLayer(input_shape=input_shape)
    self.conv1 = ConvBatchNormRelu(noof_filters=32,kernel_size=3)
    self.max_pool = MaxPool2D()
    self.conv2 = ConvBatchNormRelu(noof_filters=64,kernel_size=3)
    self.bottleneck_block1 = BottleNeckBlock(noof_filters=128,n=3)
    self.bottleneck_block2 = BottleNeckBlock(noof_filters=256,n=3)
    self.bottleneck_block3 = BottleNeckBlock(noof_filters=512,n=5)
    self.bottleneck_block4 = BottleNeckBlock(noof_filters=1024,n=5)
    self.conv_1by1 = ConvBatchNormRelu(noof_filters=noof_classes,kernel_size=1)
    self.avg_pool = GlobalAveragePooling2D()
    self.classifier = Softmax()

  def call(self,inputs):
    x = self.input_layer(inputs)
    x = self.conv1(x)
    x = self.max_pool(x)
    x = self.conv2(x)
    x = self.max_pool(x)
    x = self.bottleneck_block1(x)
    x = self.max_pool(x)
    x = self.bottleneck_block2(x)
    x = self.max_pool(x)
    x = self.bottleneck_block3(x)
    x = self.max_pool(x)
    x = self.bottleneck_block4(x)
    x = self.conv_1by1(x)
    x = self.avg_pool(x)
    x = self.classifier(x)

    return x

In [7]:
yolo_classification_model = YoloClassifier(input_shape=(None,224,224,3),noof_classes=1000)

In [8]:
yolo_classification_model.build(input_shape=(None,224,224,3))

In [9]:
yolo_classification_model.summary()

Model: "yolo_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 224, 224,   0         
                             3)]                                 
                                                                 
 conv_batch_norm_relu (ConvB  multiple                 1024      
 atchNormRelu)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 conv_batch_norm_relu_1 (Con  multiple                 18752     
 vBatchNormRelu)                                                 
                                                                 
 bottle_neck_block (BottleNe  multiple             

In [10]:
yolo_classification_model.layers[5]

In [15]:
def space_to_depth_x2(x):
    import tensorflow as tf
    return tf.nn.space_to_depth(x, block_size=2)


def space_to_depth_x2_output_shape(input_shape):
    
    return (input_shape[0], input_shape[1] // 2, input_shape[2] // 2, 4 *
            input_shape[3]) if input_shape[1] else (input_shape[0], None, None,
                                                    4 * input_shape[3])

In [17]:
class Yolo(Model):
  def __init__(self,input_shape=(None,416,416,3),noof_anchors=5,noof_classes=20):
    super().__init__()
    self.input_layer = InputLayer(input_shape=input_shape)
    self.conv1 = ConvBatchNormRelu(noof_filters=32,kernel_size=3)
    self.max_pool = MaxPool2D()
    self.conv2 = ConvBatchNormRelu(noof_filters=64,kernel_size=3)
    self.bottleneck_block1 = BottleNeckBlock(noof_filters=128,n=3)
    self.bottleneck_block2 = BottleNeckBlock(noof_filters=256,n=3)
    self.bottleneck_block3 = BottleNeckBlock(noof_filters=512,n=5)
    self.bottleneck_block4 = BottleNeckBlock(noof_filters=1024,n=5)
    self.convl1 = ConvBatchNormRelu(noof_filters=1024,kernel_size=3)
    self.convl2 = ConvBatchNormRelu(noof_filters=1024,kernel_size=3)
    self.spacial_data_to_depth=Lambda(space_to_depth_x2,output_shape=space_to_depth_x2_output_shape,name='space_to_depth')
    self.concat = Concatenate()
    self.convl3 = ConvBatchNormRelu(noof_filters=1024,kernel_size=3)
    self.output_layer = ConvBatchNormRelu(noof_filters=noof_anchors * (noof_classes + 5),kernel_size=1)

  def call(self,inputs):
    x = self.input_layer(inputs)
    x = self.conv1(x)
    x = self.max_pool(x)
    x = self.conv2(x)
    x = self.max_pool(x)
    x = self.bottleneck_block1(x)
    x = self.max_pool(x)
    x = self.bottleneck_block2(x)
    x = self.max_pool(x)
    x1 = self.bottleneck_block3(x)
    x = self.max_pool(x1)
    x = self.bottleneck_block4(x)
    x = self.convl1(x)
    x = self.convl2(x)
    y = self.spacial_data_to_depth(x1)
    z = self.concat([x,y])
    x = self.convl3(z)

    output = self.output_layer(x)

    return output


In [18]:
yolo_model = Yolo()

In [19]:
yolo_model.build(input_shape=(None,416,416,3))

In [20]:
yolo_model.summary()

Model: "yolo"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, 416, 416,   0         
                             3)]                                 
                                                                 
 conv_batch_norm_relu_19 (Co  multiple                 1024      
 nvBatchNormRelu)                                                
                                                                 
 max_pooling2d_1 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 conv_batch_norm_relu_20 (Co  multiple                 18752     
 nvBatchNormRelu)                                                
                                                                 
 bottle_neck_block_4 (Bottle  multiple                 157248 